In [1]:
from loader import train_loader, test_loader, valid_loader

In [2]:
from model import UNet

In [3]:
UNet_model = UNet()

UNet_model

UNet(
  (C1): Conv(
    (layer): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.3, inplace=False)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): Dropout(p=0.4, inplace=False)
      (7): LeakyReLU(negative_slope=0.01)
    )
  )
  (D1): DownSampling(
    (Down): Sequential(
      (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
    )
  )
  (C2): Conv(
    (layer): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.3, inplace=False)
      (3): LeakyReLU(

In [4]:
import torch
import torch.nn as nn

In [5]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [6]:
# UNet_model=UNet_model.to(DEVICE)

loss_fn= nn.BCEWithLogitsLoss()
lr=0.0001
optimizer=torch.optim.Adam(params=UNet_model.parameters(), lr=lr)
n_epochs = 10

In [7]:
from tqdm.notebook import tqdm

def train_one_epoch(model, optimizer, loss_fn, dataloader):
    model.train()
    train_loss = 0.0
    for batch in tqdm(dataloader, desc="Training"):
        images, masks = batch
        # images, masks = images.to(DEVICE), masks.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    train_loss /= len(dataloader.dataset)
    return train_loss

In [8]:
for epoch in range(n_epochs):
    train_loss = train_one_epoch(UNet_model, optimizer, loss_fn, train_loader)
    print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {train_loss:.4f}")

Training:   0%|          | 0/24 [00:00<?, ?it/s]

Epoch 1/10, Train Loss: 0.9504


Training:   0%|          | 0/24 [00:00<?, ?it/s]

Epoch 2/10, Train Loss: 0.9267


Training:   0%|          | 0/24 [00:00<?, ?it/s]

...

Epoch 8/10, Train Loss: 0.8040


Training:   0%|          | 0/24 [00:00<?, ?it/s]

Epoch 9/10, Train Loss: 0.7919


Training:   0%|          | 0/24 [00:00<?, ?it/s]

Epoch 10/10, Train Loss: 0.7815


In [10]:
# save the UNet model
torch.save(UNet_model.state_dict(), "models/UNet_model.pth")

In [6]:
from model import UNet
from torch import nn
import torch

UNet_model = UNet()

UNet_model.load_state_dict(torch.load("models/UNet_model.pth"))

loss_fn= nn.BCEWithLogitsLoss()

In [7]:
from loader import test_loader

test_loss = 0.0
UNet_model.eval()
with torch.no_grad():
    for batch in test_loader:
        images, masks = batch
        outputs = UNet_model(images)
        loss = loss_fn(outputs, masks)
        test_loss += loss.item() * images.size(0)
test_loss /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}")

Test Loss: 0.7735
